In [10]:
import os
import torch
from numpy.ma.extras import flatnotmasked_edges
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [11]:
device = torch.accelerator.current_accelerator() if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [12]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [13]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [14]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0], device='cuda:0')


In [15]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [16]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [17]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [18]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.6985, -0.3226,  0.2009, -0.1947, -0.6699,  0.3477,  0.1068,  0.2630,
         -0.4440,  0.1974,  0.0465,  0.0993,  0.3117, -0.4383, -0.3219, -0.7048,
          0.1975, -0.0331,  0.1435, -0.1460],
        [-0.3561, -0.2005,  0.3695, -0.6398, -0.7592,  0.3272, -0.1126,  0.1710,
         -0.0993, -0.3750, -0.4531, -0.0061,  0.3905, -0.4217, -0.5734, -0.0493,
          0.7544, -0.4081,  0.0057, -0.1913],
        [-0.4483,  0.2660,  0.2850, -0.2098, -0.5690,  0.5371,  0.1765, -0.0798,
         -0.2963, -0.0636, -0.2842,  0.1231,  0.2711, -0.5469, -0.5227, -0.4798,
          0.2994, -0.2970, -0.1728,  0.1217]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.2009, 0.0000, 0.0000, 0.3477, 0.1068, 0.2630, 0.0000,
         0.1974, 0.0465, 0.0993, 0.3117, 0.0000, 0.0000, 0.0000, 0.1975, 0.0000,
         0.1435, 0.0000],
        [0.0000, 0.0000, 0.3695, 0.0000, 0.0000, 0.3272, 0.0000, 0.1710, 0.0000,
         0.0000, 0.0000, 0.0000, 0.3905, 0.0000, 0.00

In [19]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [20]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [21]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0229, -0.0140, -0.0136,  ...,  0.0354, -0.0044,  0.0004],
        [-0.0311, -0.0138, -0.0007,  ...,  0.0074, -0.0216, -0.0252]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0203,  0.0081], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0377,  0.0028, -0.0218,  ...,  0.0380, -0.0321, -0.0016],
        [ 0.0236,  0.0203, -0.0118,  ...,  0.0139,  0.0103,  0.0275]],
       device='cuda:0', grad_fn=<Slice